In [ ]:
# !pip install pydub

In [ ]:
# import pydub
# import os
# import pandas as pd
# traindatabase = pd.read_csv("/content/gdrive/MyDrive/Swahili-ASR/SwahiliDataTrain.csv")
# testdatabase = pd.read_csv('/content/gdrive/MyDrive/Swahili-ASR/SwahiliDataTest.csv')

In [ ]:
# old_paths_train = traindatabase['path']
# old_paths_test = testdatabase['path']
# old_text_train = traindatabase['transcript']
# old_text_test = testdatabase['transcript']

In [ ]:
# old_paths_train[0]

In [ ]:
# from pydub import AudioSegment
# i = 1
# new_text_train = []
# new_paths_train = []
# for x,y in zip(old_paths_train,old_text_train):
#   try:
#     wav_audio = AudioSegment.from_file(x, format="wav"  )
#     filename = 'clip'+str(i)+'.mp3'
#     i+=1
#     wav_audio.export("/content/gdrive/MyDrive/Swahili-ASR/Data/DataMP3/"+filename, format="mp3")
#     new_paths_train.append("/content/gdrive/MyDrive/Swahili-ASR/Data/DataMP3/"+filename)
#     new_text_train.append(y)
#   except:
#     continue


In [ ]:
# len(new_paths_train)

In [ ]:



# wav_audio = AudioSegment.from_file("audio.wav", format="wav")
# raw_audio = AudioSegment.from_file("audio.wav", format="raw",
#                                    frame_rate=44100, channels=2, sample_width=2)

# wav_audio.export("audio1.mp3", format="mp3")
# raw_audio.export("audio2.mp3", format="mp3")

In [ ]:
%%capture
!pip install datasets==1.4.1
!pip install transformers==4.4.0
!pip install soundfile
!pip install jiwer

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': ["/content/gdrive/MyDrive/Swahili-ASR/SwahiliDataTrain.csv"],
                                              'test': '/content/gdrive/MyDrive/Swahili-ASR/SwahiliDataTest.csv'})

Using custom data configuration default-db85f6339c5a9c41


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-db85f6339c5a9c41/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c. Subsequent calls will reuse this data.


In [ ]:
dataset
datasetTrain = dataset['train']
datasetTrain = datasetTrain.remove_columns(['Unnamed: 0'])
datasetTest = dataset['test']
datasetTest = datasetTest.remove_columns(['Unnamed: 0'])

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(datasetTrain.remove_columns(["path"]))

,transcript
0,unaweza ukawa unajiuliza sasa msikilizaji mchango wa usawa wa jinsia katika maswala ya maendeleo endelevu
1,kama ndio unajiunga nasi saa hii haya ni makala
2,wapoteze nafasi zao katika chama cha nrm
3,ikiwa njiani kwenda nchi moja ya magharibi mwa afrika
4,eee ee dhahabu iliopo pale kwa hivyo kimsingi naweza kusema
5,tuna watumishi yaani wale wanao
6,wengine ni agwe awori wa eneo mbunge la
7,na kwa hivyo wananchi wengi wana wangependa labda kukaa kando
8,shukrani san msikilizaji kwa kuchangua kusikilija matangazo haya na tunaanza na taarifa ya habarii
9,shirikisho lilikua linamfumo mtazamo wa kusema


In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["transcript"] = re.sub(chars_to_ignore_regex, '', batch["transcript"]).lower() + " "
    return batch

datasetTrain = datasetTrain.map(remove_special_characters)
datasetTest = datasetTest.map(remove_special_characters)

In [ ]:
len(datasetTrain)

7126

In [ ]:
print(show_random_elements(datasetTrain.remove_columns(["path"])))

,transcript
0,mambo kadhaa ambayo yamewashawishi au yamewavuta
1,wengi kati yetu tumeshuhudia ongezeko kwa kasi kubwa
2,kwani watu wengi haswa wanawake
3,siwezi kuwafanyia mahesabu ya kuchosha
4,ramadhan machaku wa ubalozini camp area six morogoro tanzania
5,<unk> uwanja wake wa nyumbani stanford bridge
6,ya afrika mashariki na kati
7,ffu wakaona jinsi walivyovaa wakaona jinsi walivyo na mikonga na nini watu wanaogopa
8,mkutano huo ulioongozwa na mwenyeji wake ahmad zahid hamid
9,eh nadhani aah tangu mwanzo tangu


None


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcript"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = datasetTrain.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=datasetTrain.column_names)
vocab_test = datasetTest.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=datasetTest.column_names)

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 6,
 "'": 23,
 '0': 10,
 '1': 3,
 '2': 14,
 '3': 13,
 '5': 22,
 '6': 21,
 '7': 24,
 '9': 25,
 '<': 12,
 '>': 11,
 '_': 38,
 'a': 5,
 'b': 2,
 'c': 33,
 'd': 16,
 'e': 0,
 'f': 17,
 'g': 20,
 'h': 37,
 'i': 8,
 'j': 1,
 'k': 26,
 'l': 29,
 'm': 9,
 'n': 15,
 'o': 27,
 'p': 35,
 'q': 18,
 'r': 30,
 's': 32,
 't': 34,
 'u': 36,
 'v': 4,
 'w': 31,
 'x': 7,
 'y': 28,
 'z': 19}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

41

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-swahili-demo")

In [ ]:
datasetTrain[0]

{'path': '/content/gdrive/MyDrive/Swahili-ASR/Data/SWH-05-20101106/SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part10.wav',
 'transcript': 'rais wa tanzania jakaya mrisho kikwete  '}

In [ ]:
!pip install torchaudio

     |████████████████████████████████| 1.9MB 28.2MB/s 


In [ ]:
import torchaudio

def speech_file_to_array_fn(batch):
  
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["transcript"]
    return batch



In [ ]:
datasetTest = datasetTest.map(speech_file_to_array_fn, remove_columns=datasetTest.column_names)

In [ ]:
len(datasetTest)

3054

In [ ]:
datasetTest[0]

{'sampling_rate': 16000,
 'speech': [-0.0128173828125,
  -0.01287841796875,
  -0.011444091796875,
  -0.01220703125,
  -0.012115478515625,
  -0.009765625,
  -0.008697509765625,
  -0.00830078125,
  -0.0086669921875,
  -0.005126953125,
  -0.00506591796875,
  -0.004241943359375,
  -0.003936767578125,
  -0.004669189453125,
  -0.00341796875,
  -0.00469970703125,
  -0.005950927734375,
  -0.00579833984375,
  -0.005401611328125,
  -0.00616455078125,
  -0.006683349609375,
  -0.00775146484375,
  -0.00897216796875,
  -0.009765625,
  -0.01031494140625,
  -0.011444091796875,
  -0.009674072265625,
  -0.010345458984375,
  -0.009124755859375,
  -0.008148193359375,
  -0.009490966796875,
  -0.009185791015625,
  -0.00872802734375,
  -0.00592041015625,
  -0.003875732421875,
  -0.00433349609375,
  -0.00433349609375,
  -0.00225830078125,
  -0.001800537109375,
  -0.0013427734375,
  -0.000213623046875,
  0.00030517578125,
  -0.00067138671875,
  -0.000213623046875,
  -0.00054931640625,
  -0.001617431640625,
  -

In [ ]:
datasetTrain = datasetTrain.map(speech_file_to_array_fn, remove_columns=datasetTrain.column_names)


In [ ]:
# import librosa
# import numpy as np

# def resample(batch):
#     batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 48_000, 16_000)
#     batch["sampling_rate"] = 16_000
#     return batch

In [ ]:
# datasetTrain = datasetTrain.map(resample, num_proc=4)
# datasetTest = datasetTest.map(resample, num_proc=4)

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(datasetTrain)-1)

ipd.Audio(data=np.asarray(datasetTrain[rand_int]["speech"]), autoplay=True, rate=16000)

In [ ]:
rand_int = random.randint(0, len(datasetTrain)-1)

print("Target text:", datasetTrain[rand_int]["target_text"])
print("Input array shape:", np.asarray(datasetTrain[rand_int]["speech"]).shape)
print("Sampling rate:", datasetTrain[rand_int]["sampling_rate"])

Target text: na sasa tuangazie habari kutoka barani afrika ambayo kubwa hii leo  
Input array shape: (53600,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
datasetTrain = datasetTrain.map(prepare_dataset, remove_columns=datasetTrain.column_names, batch_size=8, num_proc=4, batched=True)
datasetTest= datasetTest.map(prepare_dataset, remove_columns=datasetTest.column_names, batch_size=8, num_proc=4, batched=True)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
from datasets import load_metric
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_extractor()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-swahili-demo",
  # output_dir="./wav2vec2-large-xlsr-turkish-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=100,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=datasetTrain,
    eval_dataset=datasetTest,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
100,3.428500,3.048026,1.000000,638.618000,4.782000
200,3.055300,2.872966,1.000000,644.812800,4.736000
300,2.764000,2.190374,1.000472,648.917900,4.706000
400,1.648400,1.127453,0.887868,660.656900,4.623000
500,1.055600,0.798036,0.688979,661.251700,4.619000


Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
100,3.428500,3.048026,1.000000,638.618000,4.782000
200,3.055300,2.872966,1.000000,644.812800,4.736000
300,2.764000,2.190374,1.000472,648.917900,4.706000
400,1.648400,1.127453,0.887868,660.656900,4.623000
500,1.055600,0.798036,0.688979,661.251700,4.619000
600,0.931700,0.725717,0.642261,659.739500,4.629000
700,0.783000,0.596593,0.591840,638.082700,4.786000
800,0.640400,0.561791,0.564542,635.988600,4.802000


In [ ]:
from transformers import Wav2Vec2ForCTC
model = Wav2Vec2ForCTC.from_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-swahili-demo",group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=100,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,).to("cuda")
processor = Wav2Vec2Processor.from_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-swahili-demo")

file /content/gdrive/MyDrive/wav2vec2-large-xlsr-swahili-demo/config.json not found


OSError: ignored